## TABLE OF CONTENTS:
* [Imports](#zero-bullet)
* [Parameters](#zero2-bullet)
* [Extracting Data + Pre-Processing](#first-bullet)
    * [Reading Images](#section_1_1)
    * [Cleaning & Pooling Data](#section_1_2)
    * [Splitting Data](#section_1_3)
* [Generating Deep Learning Models + Training On Data](#second-bullet)
    * [Initialize Autoencoder & Classifier](#section_2_1)
    * [Training Model & Classifying Encoded Images](#section_2_2)
* [Plot Training Metrics + Downstream Classification Results](#third-bullet)
    * [Plotting Autoencoder Train/Val Loss & Classifier Loss/Acc](#section_3_1)
    * [Plotting Confusion Matrices](#section_3_2)
* [Visualizing Decoded Images + Compressed Latent Space](#fourth-bullet)
    * [Plotting Decoded Images For Train/Test](#section_4_1)
    * [Plotting Train/Test Latent with TSNE and Isomap](#section_4_2)
* [Finding Similar Images Based On Input](#fifth-bullet)
    * [Computing KNN & Cosine Similarity In Latent Space](#section_5_1)
    * [Plotting Similar Images](#section_5_2)

## Imports <a class="anchor" id="zero-bullet"></a> </center></h1>

In [1]:
# handle imports
import os
os. chdir('/Users/ankushgupta/Documents/amazon_case_study/code')
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, UpSampling2D, Flatten, Dense, Reshape, Conv2DTranspose, InputLayer, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model, Sequential
import os
import cv2
import io
import random as rd
from sklearn.model_selection import train_test_split
from mpl_toolkits import mplot3d
from sklearn.manifold import TSNE, Isomap
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import NearestNeighbors
import sys
from skimage import io 
import argparse
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.experimental import enable_halving_search_cv  
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import GridSearchCV
%matplotlib inline

## Parameters <a class="anchor" id="zero2-bullet">

In [2]:
# read in parameters 

# for loading the data 
data_directory = "/Users/ankushgupta/Documents/amazon_case_study/data"
# for the pre-processing of the data
training_split = 0.6
validation_split = 0.2
test_split = 0.2
# for the convolutional autoencoder model 
latent_dim = 20
conv_autoencoder_lr = 0.00005
conv_autoencoder_epochs = 100
conv_autoencoder_batch = 10
# for the simple classifier model 
classifier_lr = 0.0005
classifier_epochs = 500

<h1><center> Extracting Data + Pre-Processing <a class="anchor" id="first-bullet"></a> </center></h1>

## Reading Images <a class="anchor" id="section_1_1"></a> 

In [3]:
# read the data in 
image_data_holder = {}
for directory in os.listdir(data_directory):
    sub_directory = os.path.join(data_directory, directory)
    if sub_directory.split('/')[-1].isalpha() == True:
        image_data_holder[sub_directory.split('/')[-1]] = {}
        path_to_jpgs = [os.path.join(sub_directory, _) for _ in os.listdir(sub_directory) if _.endswith(r".jpg")]
        for path in path_to_jpgs:
            image = io.imread(path)
            # try BGR2GRAY and RGB2GRAY
            # grey_scale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)[:, :, np.newaxis]
            image_data_holder[sub_directory.split('/')[-1]][path.split('/')[-1].split('.')[-2]] = image
    else:
        pass

## Cleaning & Pooling Data  <a class="anchor" id="section_1_2"></a> 

In [4]:
# intialize some information and structures
labels = list(image_data_holder.keys())
pooled_image_data = []
pooled_image_labels = []
# utilize the shape of the first image for the first folder as reference (for ease)
ref_image_dim = np.shape(image_data_holder[list(labels)[0]][list(image_data_holder[list(labels)[0]])[0]])
for global_key in labels:
    for local_key in list(image_data_holder[global_key].keys()):
        # normalize the image between 0 and 1 
        image_data_holder[global_key][local_key] = image_data_holder[global_key][local_key] / np.max(image_data_holder[global_key][local_key])
        # making sure that all of the images are the same size 
        dims_check = image_data_holder[global_key][local_key].shape
        if dims_check != ref_image_dim:
            print('Resized Image: [' + global_key + '][' + local_key + ']')
            image_data_holder[global_key][local_key] = cv2.resize(image_data_holder[global_key][local_key], ref_image_dim[:2], interpolation=cv2.INTER_LINEAR)
        # pool data together for downstream tasks 
        pooled_image_data.append(image_data_holder[global_key][local_key])
        pooled_image_labels.append(np.where(np.asarray(labels) == np.asarray(global_key))[0][0])

IndexError: list index out of range

## Splitting Data <a class="anchor" id="section_1_3"></a> 

In [27]:
# split the data into training/validation/test and reshape to input into model
x_train, x_temp, y_train, y_temp = train_test_split(np.asarray(pooled_image_data), np.asarray(pooled_image_labels), test_size=1-training_split, random_state=42)
x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=1 / ((1-training_split) / test_split), random_state=42)

training_data = np.array(x_train.reshape(-1, x_train.shape[1], x_train.shape[2], x_train.shape[-1]))
validation_data = np.array(x_val.reshape(-1, x_val.shape[1], x_val.shape[2], x_val.shape[-1]))
test_data = np.array(x_test.reshape(-1, x_test.shape[1], x_test.shape[2], x_test.shape[-1]))
full_data = np.array(np.asarray(pooled_image_data).reshape(-1, np.asarray(pooled_image_data).shape[1], np.asarray(pooled_image_data).shape[2], np.asarray(pooled_image_data).shape[-1]))

<h1><center> Generating Deep Learning Models + Training On Data <a class="anchor" id="second-bullet"></a> </center></h1>

## Initialize Autoencoder & Classifier <a class="anchor" id="section_2_1"></a>

In [28]:
def create_convolutional_autoencoder():
    # initialize the convolutional autoencoder for image compression ---------> 
    encoder_input = Input(shape= ref_image_dim, name='Input Layer')
    encoder_1 = Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation='relu')(encoder_input)
    encoder_2 = Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu')(encoder_1)
    encoder_3 = Flatten()(encoder_2)
    encoder_4 = Dense(latent_dim)(encoder_3)
    encoder = Model(encoder_input, encoder_4, name='encoder')
    encoder.summary()

    decoder_input = Input(shape=(latent_dim,))
    decoder_1 = Dense(units=7*7*32, activation=tf.nn.relu)(decoder_input)
    decoder_2 = Reshape(target_shape=(7, 7, 32))(decoder_1)
    decoder_3 = Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='relu')(decoder_2)
    decoder_4 = Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='relu')(decoder_3)
    decoder_5 = Conv2DTranspose(filters=ref_image_dim[-1], kernel_size=3, strides=1, padding='same')(decoder_4)
    decoder = Model(decoder_input, decoder_5, name='decoder')
    decoder.summary()

    encoded = encoder(encoder_input)
    decoded = decoder(encoded)
    convolutional_autoencoder = Model(inputs=encoder_input, outputs=decoded, name='autoencoder')
    
    if optimizer == 'SGD': 
        optimizer = tf.keras.optimizers.SGD(lr=learning_rate)
    elif optimizer == 'Adam': 
        optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
    convolutional_autoencoder.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=conv_autoencoder_lr), loss='mse')
    # convolutional_autoencoder.summary()
    return convolutional_autoencoder

def create_classifier():
    # downstream classifier for gauging how well the encoded representation of original input can classify for the different labels
    classifier_input = Input(shape=(latent_dim,))
    classifier_1 = Dense(np.round(latent_dim/2), activation='relu')(classifier_input)
    classifier_2 = Dense(np.round(latent_dim/4), activation='relu')(classifier_1)
    classifier_3 = Dense(6, activation='softmax')(classifier_2)
    downstream_classifier = Model(classifier_input, classifier_3)
    downstream_classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=classifier_lr), loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                             metrics=['accuracy'])
    downstream_classifier.summary()
    return downstream_classifier

In [29]:
epochs = [10, 50, 100]
batch_size = [10, 20, 40, 60, 80, 100, 200, 500, 1000]
optimizer = ['SGD', 'Adam']
learning_rate = [1e-1, 0.5e-1, 1e-2, 0.5e-2, 1e-3, 0.5e-3, 1e-4, 0.5e-4, 1e-5, 0.5e-5, 1e-6, 0.5e-6, 1e-7, 0.5e-7]
param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer, learning_rate=learning_rate)

model = KerasRegressor(build_fn=create_convolutional_autoencoder, verbose=1)
grid = HalvingGridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train, x_train)

/var/folders/d6/6myqdq356m3__krcnz6h3wbm0000gn/T/ipykernel_6473/2423895376.py:5: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasRegressor(build_fn=create_convolutional_autoencoder, verbose=1)
2022-03-30 16:19:41.533171: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-30 16:19:41.533209: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input Layer (InputLayer)    [(None, 28, 28, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 13, 13, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 6, 6, 64)          18496     
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 20)                46100     
                                                                 
Total params: 65,492
Trainable params: 65,492
Non-trainable params: 0
_________________________________________________________________
Model: "decoder"
______________________________________

Total params: 70,755
Trainable params: 70,755
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
 1/67 [..............................] - ETA: 1:02 - loss: 0.4177Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input Layer (InputLayer)    [(None, 28, 28, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 13, 13, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 6, 6, 64)          18496     
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 20)                46100     
                                             

 1/67 [..............................] - ETA: 1:03 - loss: 0.4259Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input Layer (InputLayer)    [(None, 28, 28, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 13, 13, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 6, 6, 64)          18496     
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 20)                46100     
                                                                 
Total params: 65,492
Trainable params: 65,492
Non-trainable params: 0
________________________________________________________

134/134 [==============================] - 7s 52ms/step - loss: 0.0209
Epoch 6/100
134/134 [==============================] - 7s 49ms/step - loss: 0.0200
Epoch 7/10
134/134 [==============================] - 7s 51ms/step - loss: 0.0191
Epoch 9/100
67/67 [==============================] - 6s 91ms/step - loss: 0.0204
67//134 [===============>..............] - ETA: 2s - loss: 0.01867 [==============================] - 6s 91ms/step - loss: 0.0204
134/134 [==============================] - 6s 46ms/step - loss: 0.0189.............] - ETA: 4s - loss: 0.016
Epoch 10/10
134/134 [==============================] - 6s 42ms/step - loss: 0.0186
Epoch 11/100
134/134 [==============================] - 6s 42ms/step - loss: 0.0187
Epoch 11/100
 18/134 [===>..........................] - ETA: 2s - loss: 0.0186Model: "encoder"
_________________________________________________________________
 29/134 [=====>........................] - ETA: 2s - loss: 0.01813)]       0         
                              

...

67/67 [==============================] - 6s 94ms/step - loss: 0.0122
Epoch 67/100
134/134 [==============================] - 8s 61ms/step - loss: 0.0097================>..........] - ETA: 2s - loss: 0.01..................] - ETA: 5s - loss: 0.024
Epoch 70/100
10/67 [===>..........................] - ETA: 6s - loss: 0.0120Epoch 14/100


34/34 [==============================] - 6s 166ms/step - loss: 0.0230
Epoch 71/100
34/34 [==============================] - 5s 148ms/step - loss: 0.0192
Epoch 28/50
134/134 [==============================] - 8s 61ms/step - loss: 0.0092
Epoch 73/100
50/67 [=====================>........] - ETA: 1s - loss: 0.0107Epoch 74/100


134/134 [==============================] - 8s 62ms/step - loss: 0.0095
Epoch 75/100
67/67 [==============================] - 7s 103ms/step - loss: 0.0107 - loss: 0.008
Epoch 75/100
34/34 [==============================] - 5s 159ms/step - loss: 0.0190
Epoch 75/100
34/34 [==============================] - 5s 151ms/step - loss: 0.0191
Epoch 21/100
134/134 [==============================] - 8s 58ms/step - loss: 0.00920
Epoch 76/100
134/134 [==============================] - 9s 65ms/step - loss: 0.0091
Epoch 35/50
67/67 [==============================] - 7s 106ms/step - loss: 0.0104
Epoch 79/100
48/67 [====================>.........] - ETA: 1s - loss: 0.0102Epoch 82/100


134/134 [==============================] - 8s 62ms/step - loss: 0.0090
Epoch 82/100
134/134 [==============================] - 9s 64ms/step - loss: 0.0089
Epoch 83/100
33/67 [=============>................] - ETA: 3s - loss: 0.01005

KeyboardInterrupt: 

## Training Model & Classifying Encoded Images <a class="anchor" id="section_2_2"></a>

In [ ]:
# run the data through the model
reconstruction_metrics = convolutional_autoencoder.fit(training_data, training_data, epochs=conv_autoencoder_epochs, 
                                                       batch_size=conv_autoencoder_batch, verbose=1, shuffle=True, validation_data=(validation_data, validation_data))
encoded_training_set = encoder.predict(training_data)
encoded_test_set = encoder.predict(test_data)
encoded_full_set = encoder.predict(full_data)
classification_metrics = downstream_classifier.fit(encoded_training_set, y_train, epochs=classifier_epochs, verbose=1, shuffle=True)

Epoch 1/100
1800/1800 [==============================] - 18s 10ms/step - loss: 0.0390 - val_loss: 0.0179
Epoch 2/100
1800/1800 [==============================] - 18s 10ms/step - loss: 0.0161 - val_loss: 0.0143
Epoch 3/100
1800/1800 [==============================] - 18s 10ms/step - loss: 0.0132 - val_loss: 0.0117
Epoch 4/100
1800/1800 [==============================] - 17s 10ms/step - loss: 0.0108 - val_loss: 0.0101
Epoch 5/100
1800/1800 [==============================] - 18s 10ms/step - loss: 0.0099 - val_loss: 0.0095
Epoch 6/100
1800/1800 [==============================] - 18s 10ms/step - loss: 0.0094 - val_loss: 0.0092
Epoch 7/100
1800/1800 [==============================] - 18s 10ms/step - loss: 0.0092 - val_loss: 0.0090
Epoch 8/100
1800/1800 [==============================] - 18s 10ms/step - loss: 0.0089 - val_loss: 0.0088
Epoch 9/100
1800/1800 [==============================] - 18s 10ms/step - loss: 0.0088 - val_loss: 0.0087
Epoch 10/100
1800/1800 [==============================]

<h1><center> Plot Training Metrics + Downstream Classification Results <a class="anchor" id="third-bullet"></a> </center></h1>

## Plotting Autoencoder Train/Val Loss & Classifier Loss/Acc <a class="anchor" id="section_3_1"></a>

In [ ]:
# plot the training and validation loss metrics 
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(30, 12))

# plot the image reconstruction loss 
ax1.plot(range(len(reconstruction_metrics.history['loss'])), reconstruction_metrics.history['loss'], label='Training Loss', linewidth=5, color='dodgerblue')
ax1.plot(range(len(reconstruction_metrics.history['val_loss'])), reconstruction_metrics.history['val_loss'], label='Validation Loss', linewidth=5, color='red')
ax1.legend(fontsize=14), ax1.set_xlabel('Epochs', fontsize=14), ax1.set_ylabel('Mean Squared Error (MSE)', fontsize=14)
ax1.set_title('Image Reconstruction Loss', fontsize=16)

# plot the classification metrics 
ax2.plot(range(len(classification_metrics.history['loss'])), classification_metrics.history['loss'], label='Training Loss', linewidth=5, color='dodgerblue')
ax3.plot(range(len(classification_metrics.history['accuracy'])), classification_metrics.history['accuracy'], label='Training Accuracy', linewidth=5, color='green')
ax2.legend(fontsize=14), ax2.set_xlabel('Epochs', fontsize=14), ax2.set_ylabel('Categorical Cross Entropy', fontsize=14)
ax3.legend(fontsize=14), ax3.set_xlabel('Epochs', fontsize=14), ax3.set_ylabel('Accuracy', fontsize=14)
ax2.set_title('Classification Loss On Encoded Images', fontsize=16)
ax3.set_title('Classification Accuracy on Encoded Images', fontsize=16)

plt.suptitle('Metrics From Autoencoder and Classifier', fontsize=22, fontweight='bold')
plt.show()

## Plotting Confusion Matrices <a class="anchor" id="section_3_2"></a>

In [ ]:
# plot confusion matrix of the classification results 

fig, axes = plt.subplots(1, 2, figsize=(20, 8), sharey=True)

train_predictions = downstream_classifier.predict(encoded_training_set)
train_predicted_labels = [np.where(train_predictions[x] == np.max(train_predictions[x]))[0][0] for x in range(np.shape(train_predictions)[0])]
train_norm_matrix = confusion_matrix(y_train, train_predicted_labels, normalize='true')
sns.heatmap(train_norm_matrix, ax=axes[0], annot=True, yticklabels=labels, xticklabels=labels)
axes[0].set_title('Encoded Training Data', fontsize=16)

test_predictions = downstream_classifier.predict(encoded_test_set)
test_predicted_labels = [np.where(test_predictions[x] == np.max(test_predictions[x]))[0][0] for x in range(np.shape(test_predictions)[0])]
test_norm_matrix = confusion_matrix(y_test, test_predicted_labels, normalize='true')
sns.heatmap(test_norm_matrix, ax=axes[1], annot=True, yticklabels=labels, xticklabels=labels)
axes[1].set_title('Encoded Test Data', fontsize=16)

plt.suptitle('Confusion Matrices From Encoded Training/Test Data', fontweight='bold', fontsize=20)
plt.show()

<h1><center> Visualizing Decoded Images + Compressed Latent Space <a class="anchor" id="fourth-bullet"></a> </center></h1>

## Plotting Decoded Images For Train/Test <a class="anchor" id="section_4_1"></a>

In [ ]:
# utilize the encoder to generate latent representations on training set and test set 
test_decoded = convolutional_autoencoder.predict(test_data)
train_decoded = convolutional_autoencoder.predict(training_data)
random_image = np.random.randint(0, np.shape(test_decoded)[0]-1)
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(10, 10))
ax1.imshow((train_decoded[random_image]* 255).astype(np.uint8))
ax2.imshow((training_data[random_image]* 255).astype(np.uint8))
ax3.imshow((test_decoded[random_image]* 255).astype(np.uint8))
ax4.imshow((test_data[random_image]* 255).astype(np.uint8))
ax1.set_title('Training Decoded'), ax2.set_title('Original Training'), ax3.set_title('Test Decoded'), ax4.set_title('Original Test')
plt.suptitle('Decoded Images For Training & Test', fontweight='bold', fontsize=16)
plt.show()

## Plotting Train/Test Latent [TSNE & Isomap] <a class="anchor" id="section_4_2"></a>

In [ ]:
# plot the latent space 

if latent_dim == 2 or latent_dim == 3:
    if latent_dim == 3:
        fig = plt.figure(figsize=(20, 10))
        ax1 = fig.add_subplot(1, 2, 1, projection='3d')
        ax2 = fig.add_subplot(1, 2, 2, projection='3d')
    elif latent_dim == 2:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    colors = ['blue', 'red', 'green', 'yellow', 'black', 'orange']
    for num in np.unique(y_train):
        train_indices = np.where(y_train == num)[0]
        test_indices = np.where(y_test == num)[0]
        if latent_dim == 3:
            ax1.scatter3D(encoded_training_set[train_indices, 0], encoded_training_set[train_indices, 1], encoded_training_set[train_indices, 2], alpha=0.5, color=colors[num])
            ax2.scatter3D(encoded_test_set[test_indices, 0], encoded_test_set[test_indices, 1], encoded_test_set[test_indices, 2], alpha=0.5, color=colors[num])
        elif latent_dim == 2:
            ax1.scatter(encoded_training_set[train_indices, 0], encoded_training_set[train_indices, 1], alpha=0.5, color=colors[num])
            ax2.scatter(encoded_test_set[test_indices, 0], encoded_test_set[test_indices, 1], alpha=0.5, color=colors[num])
    ax1.set_title('Training Data'), ax2.set_title('Test Data')
    if latent_dim >= 2:
        ax1.set_xlabel('Latent 1'), ax1.set_ylabel('Latent 2'), ax2.set_xlabel('Latent 1'), ax2.set_ylabel('Latent 2')
    elif latent_dim >= 3:
        ax1.set_zlabel('Latent 3'), ax2.set_zlabel('Latent 3')
    plt.suptitle(str(latent_dim) + 'D Latent Space', fontsize=16, fontweight='bold')
elif latent_dim > 3: 
    # using t-SNE and Isomap on high dimensional latent space for visualization
    # for training data 
    TSNE_model = TSNE(n_components=2, verbose=1, random_state=123)
    isomap_model = Isomap(n_components=2)
    tsne_training = TSNE_model.fit_transform(encoded_training_set)
    isomap_training = isomap_model.fit_transform(encoded_training_set)
    # for test data
    TSNE_model = TSNE(n_components=2, verbose=1, random_state=123)
    isomap_model = Isomap(n_components=2)
    tsne_test = TSNE_model.fit_transform(encoded_test_set)
    isomap_test = isomap_model.fit_transform(encoded_test_set)
    # plotting t-sne and isomap results for training and test datasets 
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 20))
    colors = ['blue', 'red', 'green', 'yellow', 'black', 'orange']
    for x in np.unique(y_train):
        training_indices = np.where(y_train == x)[0]
        test_indices = np.where(y_test == x)[0]
        ax1.scatter(tsne_training[training_indices, 0], tsne_training[training_indices, 1], color=colors[x], label=str(x))
        ax2.scatter(isomap_training[training_indices, 0], isomap_training[training_indices, 1], color=colors[x], label=str(x))
        ax3.scatter(tsne_test[test_indices, 0], tsne_test[test_indices, 1], color=colors[x], label=str(x))
        ax4.scatter(isomap_test[test_indices, 0], isomap_test[test_indices, 1], color=colors[x], label=str(x))

    titles = ['TSNE [Training Data]', 'Isomap [Training Data]', 'TSNE [Test Data]', 'Isomap [Test Data]']
    axes = [ax1, ax2, ax3, ax4]
    for axis in range(len(axes)):
        axes[axis].set_xlabel('Latent 1', fontsize=16), axes[axis].set_ylabel('Latent 2', fontsize=16), axes[axis].legend(fontsize=14), axes[axis].set_title(titles[axis], fontsize=20)
    plt.suptitle('Manifold Learning On Convolutional Autoencoder Latent Space', fontsize=22, fontweight='bold')
plt.show()

<h1><center> Finding Similar Images Based On Input <a class="anchor" id="fifth-bullet"></a> </center></h1>

## Computing KNN & Cosine Similarity In Latent Space <a class="anchor" id="section_5_1"></a> 

In [ ]:
# generate cosine similarity analysis and KNN for full dataset encoding 
image_input_index = np.random.randint(0, len(encoded_full_set))
top_similar_images = 20
if top_similar_images+1 > 6:
    top_images_for_plotting = 5
else:
    top_images_for_plotting = top_similar_images+1

# KNN 
X = np.array(encoded_full_set)
nbrs = NearestNeighbors(n_neighbors=top_similar_images+1).fit(X)
distances, indices = nbrs.kneighbors(X)
knn_indices = indices[image_input_index, :]

# cosine similarity
encoded_full_df = pd.DataFrame(data=encoded_full_set)
cosine_vals = cosine_similarity(encoded_full_df, dense_output=True)
cosine_indices = np.argpartition(cosine_vals[image_input_index, :], -1*top_similar_images-1)[-1*top_similar_images-1:]
cosine_indices = list(cosine_indices)
cosine_indices.pop(cosine_indices.index(image_input_index))
cosine_indices.insert(0, image_input_index)

# overlap 
overlap_indices = list(set(cosine_indices)&set(knn_indices))
overlap_indices.pop(overlap_indices.index(image_input_index))

## Plotting Similar Images <a class="anchor" id="section_5_2"></a> 

In [ ]:
# initialize the plots 
fig, axes1 = plt.subplots(1, top_images_for_plotting, figsize=(25, 5))
plt.suptitle('Nearest Neighbors With Minkowski Distance', fontweight='bold', fontsize=20)
fig2, axes2 = plt.subplots(1, top_images_for_plotting, figsize=(25, 5))
plt.suptitle('Cosine Metric', fontweight='bold', fontsize=20)

# figure for overlap between the two sets ---> images with strong similarity
if overlap_indices:
    fig3, axes3 = plt.subplots(1, top_images_for_plotting, figsize=(25, 5))
    plt.suptitle('KNN and Cosine Overlap', fontweight='bold', fontsize=20)
    iterations = 3
else:
    iterations = 2

for x in range(iterations):
    if x == 0:
        index_list = knn_indices[:top_images_for_plotting]
        axes = axes1 
    elif x == 1:
        index_list = cosine_indices[:top_images_for_plotting]
        axes = axes2
    elif x == 2:
        index_list = overlap_indices[:top_images_for_plotting]
        axes = axes3
    for index in range(len(index_list)):
        axes[index].imshow(full_data[index_list[index]])
        if x == 0 or x == 1:
            if index == 0:
                axes[index].set_title('Input Image [' + str(index_list[index]) + ']')
            else:
                axes[index].set_title(str(index_list[index]))
        elif x == 2:
            for _ in range(len(overlap_indices), top_images_for_plotting):
                axes3[_].set_axis_off()
            axes[index].set_title(str(index_list[index]))
plt.show()
print('# Of Similar Image Overlap Between NN and Cosine = ' + str(len(overlap_indices)) + '/' + str(top_similar_images))